In [1]:
import os
import re
import copy
import sklearn
import pandas as pd
import numpy as np
from project_utilities import import_train_data, import_test_data
from sklearn import model_selection, feature_extraction, preprocessing, svm, pipeline, metrics, tree, linear_model

In [2]:
os.chdir(r'C:\Users\35904\Desktop\Mcgill_Study\551ML\project2\sklearn_code')
path = os.getcwd()

path_pos_word = os.path.join(path,'positive-words.txt')
path_neg_word = os.path.join(path,'negative-words.txt')

pos_word = []
neg_word = []

with open (path_pos_word, encoding='utf8') as fd:
    pos_word = fd.read()
with open (path_neg_word, encoding='utf8') as fd:
    neg_word = fd.read()

#pos_word = pd.DataFrame(repr(pos_word).split('\\'))
#neg_word = pd.DataFrame(repr(neg_word).split('\\'))

pos_word = repr(pos_word).split('\\') # length is 2006
neg_word = repr(neg_word).split('\\') # length is 4783

print(pos_word[10])
print(neg_word[100])

In [3]:
os.chdir(r'C:\Users\35904\Desktop\Mcgill_Study\551ML\project2\train\train')
path = os.getcwd()
row_data = import_train_data(path)

os.chdir(r'C:\Users\35904\Desktop\Mcgill_Study\551ML\project2\test\test')
path = os.getcwd()
test_data = import_test_data(path)

print(row_data[:2])
print(test_data[:2])
#row_data = pd.DataFrame(row_data)
#test_data = pd.DataFrame(test_data)

In [8]:
def delete_item(data_input, item_name):
    row_data = copy.deepcopy(data_input)
    for index, value in enumerate (row_data):
        del row_data[index][item_name]
    return row_data

def get_keyvalue (input_data, key):
    keyvalue = [0]*len(input_data)
    for index, value in enumerate (input_data):
        keyvalue[index] = value[key]
    return keyvalue

def occurrence(data_point, words):
    occurrence = [0]*len(words)
    for index, value in enumerate (words):
        occurrence[index] = int (value in data_point)
    return occurrence

In [9]:
text = delete_item(row_data, 'category')
category = delete_item(row_data, 'text')
print(text[10])
print(category[20])

text_only = get_keyvalue(text,'text')
category_only = get_keyvalue(category, 'category')

print(text_only[10])
print(category_only[20])

In [14]:
X_train, X_validation, y_train, y_validation = model_selection.train_test_split(
    text_only,
    category_only,
    test_size=0.2)

In [41]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
class Sentament_words_occurence (BaseEstimator, TransformerMixin):
    
    def fit(self, x, y=None):
        return self

    def transform(self, input_data):

        text = copy.deepcopy(input_data)
        #sentament_feature = np.recarray(shape =(len(text),len(pos_word) +len(neg_word)),
        #                                dtype = [('Nothing',object)])
        sentament_feature = np.zeros(shape=(len(input_data),len(pos_word) +len(neg_word)))
        for index_text, text_content in enumerate(input_data):
            pos_words_occurence = occurrence(text_content, pos_word)
            neg_words_occurence = occurrence(text_content, neg_word)
            words_occurence = pos_words_occurence + neg_words_occurence
            for index_occurence, value_occurence in enumerate(words_occurence):
                sentament_feature[index_text][index_occurence] = value_occurence

        return sentament_feature

In [42]:
occurence_vect = Sentament_words_occurence().fit(X_train)
X_train_counts = occurence_vect.transform(X_train)
X_test_counts = occurence_vect.transform(X_validation)
print(X_train_counts.shape)
print(X_test_counts.shape)

(5000, 6789)

In [61]:
X_train_counts.shape

(20000, 6789)

In [62]:
20000*6789

135780000

In [60]:
sum(X_train_counts.sum(axis=0))/6789

3.4432169686257179

In [43]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(20000, 6789)
(5000, 6789)


In [44]:
from sklearn.preprocessing import Normalizer

normalizer_tranformer = Normalizer().fit(X=X_train_tfidf)
X_train_normalized = normalizer_tranformer.transform(X_train_tfidf)
X_test_normalized = normalizer_tranformer.transform(X_test_tfidf)
print(X_train_normalized.shape)
print(X_test_normalized.shape)

(20000, 6789)
(5000, 6789)


In [54]:
X_train_normalized[:100]

<100x6789 sparse matrix of type '<class 'numpy.float64'>'
	with 113 stored elements in Compressed Sparse Row format>

In [45]:
from sklearn import metrics
clf = svm.LinearSVC().fit(X_train_normalized, y_train)
y_pred = clf.predict(X_test_normalized)
print(metrics.classification_report(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.74      0.64      2504
           1       0.62      0.42      0.50      2496

   micro avg       0.58      0.58      0.58      5000
   macro avg       0.59      0.58      0.57      5000
weighted avg       0.59      0.58      0.57      5000



## pipeline

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline

occurence_pipeline =  Pipeline([
    ('occurence', Sentament_words_occurence()),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf', svm.LinearSVC()),
])

In [52]:
occurence_pipeline.fit(X_train, y= y_train)
y_pred = occurence_pipeline.predict(X_validation)
print(metrics.classification_report(y_validation, y_pred))

KeyboardInterrupt: 